In [1]:
model_name = 'deep_tree1'
data_dir = './data/mlboot_dataset/'
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import scipy.sparse as sp
import gc

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
df = pd.read_csv(data_dir + 'preprocessed.csv') 
q = pd.read_csv(data_dir + 'sessions.csv')
df = df.merge(q, on='uid', how='left')
del q
y = pd.read_table(data_dir + 'mlboot_train_answers.tsv')
y.columns = ['uid','target']
df = df.merge(y, on='uid', how='left')

df_train_index = df[~df.target.isnull()].index
df_test_index = df[df.target.isnull()].index

mat = sp.load_npz(data_dir+'dmat1.npz').tolil()

train_mat = mat[df_train_index.tolist()]
test_mat = mat[df_test_index.tolist()]
mat = mat.tocsc()[:, np.where((train_mat.getnnz(axis=0) > 1) & (test_mat.getnnz(axis=0) > 1))[0]].tocsr()

In [3]:
train_mat = mat[df_train_index.tolist()]
test_mat = mat[df_test_index.tolist()]
del mat
gc.collect()

0

In [4]:
X = df.loc[~df.target.isnull(),:].reset_index(drop=True)
x_te = df.loc[df.target.isnull(),:].reset_index(drop=True)

In [5]:
train_cols = [
       'sess_keys_mean','sess_keys_max','diff_key1_mean','diff_key1_max','diff_key2_mean',
       'diff_key2_max','diff_key3_mean','diff_key3_max','quot_key1_mean','quot_key1_max',
       'quot_key2_mean','quot_key2_max','quot_key3_mean','quot_key3_max',
       u'num_keys_f1',
       u'sum_values_f1', u'num_keys_f2', u'sum_values_f2', u'num_keys_f3',
       u'sum_values_f3', u'num_times_cat_eq_0', u'num_times_cat_eq_1',
       u'num_times_cat_eq_2', u'num_times_cat_eq_3', u'num_times_cat_eq_4',
       u'num_times_cat_eq_5', u'records', u'max_days', u'min_days',
       u'sum_values_f1_std', u'num_keys_f1_std', u'sum_values_f2_std',
       u'num_keys_f2_std', u'sum_values_f3_std', u'num_keys_f3_std',
       u'sum_values_f1_max', u'num_keys_f1_max', u'sum_values_f2_max',
       u'num_keys_f2_max', u'sum_values_f3_max', u'num_keys_f3_max',
       u'sum_values_f1_mean', u'num_keys_f1_mean', u'sum_values_f2_mean',
       u'num_keys_f2_mean', u'sum_values_f3_mean', u'num_keys_f3_mean']

# Train the model
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 128,
    #'max_depth' : 12,
    #'min_data' : 30,
    'lambda_l2' : 41.5,
    #'min_sum_hessian_in_leaf' : 0.3,
    'max_bin': 64,
    #'max_drop' : 1,
    'learning_rate': 0.01,
    'feature_fraction': 0.21,
    'verbose': 0
}

kf = KFold(n_splits=5, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

for train_index,test_index in kf.split(X):       
    print('fold', ifold)
    
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va = X.loc[train_index, train_cols].values,X.loc[test_index, train_cols].values
    X_te = x_te[train_cols]
    
    print('prepare train')
    X_tr = sp.hstack([
        X_tr, train_mat[train_index]
    ]).tocsr()
    print('prepare valid')
    X_va = sp.hstack([
        X_va, train_mat[test_index]
    ]).tocsr()
    
    # Create the LightGBM data containers
    tr_data = lgb.Dataset(X_tr, label=y_tr) #, categorical_feature=cate_cols
    va_data = lgb.Dataset(X_va, label=y_va) #, categorical_feature=cate_cols

    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=va_data,
                      num_boost_round=8000,
                      early_stopping_rounds=200,
                      verbose_eval=50)
    
    yhat = model.predict(X_va, model.best_iteration)
    print(ifold,roc_auc_score(y_va,yhat))
    scores.append(roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    X_te = sp.hstack([
        X_te, test_mat
    ]).tocsr()   
    
    ytst = model.predict(X_te, model.best_iteration)
    y_pred += ytst*0.1
    
    ifold += 1

fold 0
prepare train
prepare valid
Training until validation scores don't improve for 200 rounds.
[50]	valid_0's auc: 0.612087
[100]	valid_0's auc: 0.617674
[150]	valid_0's auc: 0.628773
[200]	valid_0's auc: 0.638404
[250]	valid_0's auc: 0.645776
[300]	valid_0's auc: 0.652319
[350]	valid_0's auc: 0.656895
[400]	valid_0's auc: 0.660399
[450]	valid_0's auc: 0.663475
[500]	valid_0's auc: 0.665527
[550]	valid_0's auc: 0.667765
[600]	valid_0's auc: 0.669699
[650]	valid_0's auc: 0.670845
[700]	valid_0's auc: 0.671992
[750]	valid_0's auc: 0.673245
[800]	valid_0's auc: 0.674074
[850]	valid_0's auc: 0.675134
[900]	valid_0's auc: 0.675822
[950]	valid_0's auc: 0.6765
[1000]	valid_0's auc: 0.676953
[1050]	valid_0's auc: 0.677339
[1100]	valid_0's auc: 0.677511
[1150]	valid_0's auc: 0.677769
[1200]	valid_0's auc: 0.677967
[1250]	valid_0's auc: 0.677978
[1300]	valid_0's auc: 0.678031
[1350]	valid_0's auc: 0.677949
[1400]	valid_0's auc: 0.677986
[1450]	valid_0's auc: 0.677992
Early stopping, best iter

In [6]:
print(scores,np.mean(scores))
roc_auc_score(X.target.values, y_oof.target.values)

[0.67808186306710017, 0.67606623010173095, 0.67439949783524289, 0.68215942368185345, 0.67994332442161087] 0.678130067822


0.67802247841117547

In [7]:
results_dir = './results/'
np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

In [8]:
sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub.head()

isnull? False


,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.126866
1,ac4b8244f3ae82df511b002257473c11,0.026449
2,483d8b91e49522c8a5bbe37f3872c749,0.058698
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.017844
4,fdbfba9842ff0bf86d600eb334c7c42b,0.017396


In [9]:
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)